In [1]:
# import wget
# for i in range(1, 8):
#     for letter in ['c', 'w', 'p', 's']:
#         url = f'https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/p0{i}_{letter}.txt'
#         wget.download(url, 'D:\Study\AIO\Lab2\\benchmarks')

In [2]:
def count_weight(indexes: list, weights: list):
    '''Counts total weight of knapsack'''
    weight = 0
    assert len(indexes) == len(weights)
    for i in range(len(weights)):
        weight += weights[i] * indexes[i]
    return weight

In [3]:
import numpy as np

def knapsack(method, cap, p, w):
    '''Returns array of items (1 if item is taken, else 0), time and amount of operations
        of method'''
    times = []
    for i in range(10):
        import time
        start = time.time()
        indexes, profit, ops_cnt = method(cap, p, w)
        end = time.time()
        time = end - start
        times.append(time)
    return indexes, profit, ops_cnt, np.mean(times)

In [4]:
def dp(cap: int, p: list, w: list):
    '''DP on weights'''
    n = len(w)
    dp = [[0 for i in range(cap + 1)] for i in range(n + 1)]
    items = []
    ops_cnt = 0
    for i in range(1, n + 1):
        for j in range(1, cap + 1):
            wi = w[i - 1]
            if wi <= j:
                dp[i][j] = max(dp[i - 1][j], p[i - 1] + dp[i - 1][j - wi])
            else:
                dp[i][j] = dp[i - 1][j]
            ops_cnt += 1
                
    def findAns(dp: list, i: int, j: int, w: list):
        if dp[i][j] == 0: 
            return
        if dp[i - 1][j] == dp[i][j]:
            findAns(dp, i - 1, j, w)
        else:
            findAns(dp, i - 1, j - w[i - 1], w)
            items.append(i - 1)
            
    findAns(dp, n, cap, w)        
    indexes = []
    for i in range(n):
        if i in items:
            indexes.append(1)
        else:
            indexes.append(0)
    
    return indexes, dp[n][cap], ops_cnt

In [5]:
def two_approx(cap: int, p: list, w: list):
    """2-approx algorithm for Knapsack"""
    #greed heuristic
    n = len(p)
    ops_cnt = 0
    p_sorted = []
    for i, item in enumerate(p):
        ops_cnt += 1
        p_sorted.append((i, item))
    p_sorted = sorted(p_sorted, key=lambda x: x[1], reverse=True)
    ops_cnt += int(n * np.log2(n))
    greed_total_profit = 0
    remaining_cap = cap
    greed_items = []
    for i, profit in p_sorted:
        ops_cnt += 1
        if remaining_cap - w[i] < 0:
            break
        greed_total_profit += profit
        remaining_cap -= w[i]
        greed_items.append(i)
      
    #maxgreed heuristic  
    pw_sorted = []
    for i in range(n):
        ops_cnt += 1
        pw_sorted.append((i, p[i]/w[i]))
    pw_sorted = sorted(pw_sorted, key=lambda x: x[1], reverse=True)
    ops_cnt += int(n * np.log2(n))
    maxgreed_total_profit = 0
    remaining_cap = cap
    maxgreed_items = []
    for i, utility in pw_sorted:
        ops_cnt += 1
        if remaining_cap - w[i] < 0:
            break
        maxgreed_total_profit += p[i]
        remaining_cap -= w[i]
        maxgreed_items.append(i)
        
    max_profit = 0
    best_items = []
    if greed_total_profit >= maxgreed_total_profit:
        max_profit = greed_total_profit
        best_items = greed_items
    else:
        max_profit = maxgreed_total_profit
        best_items = maxgreed_items
    
    indexes = []
    for i in range(n):
        if i in best_items:
            indexes.append(1)
        else:
            indexes.append(0)
        
    return indexes, max_profit, ops_cnt

In [6]:
from itertools import chain, combinations

def ptas(cap, p, w, k : int = 7):
    '''Knapsack task via PTAS algo by Sahni'''
    n_items = len(w)
    ops = 0

    def powerset_k(seq, k = k):
        return chain.from_iterable(combinations(seq, r) for r in range(0, min(k, n_items)+1))

    def GS(subset):
        '''Greedy Search procedure'''
        ops_ = 0
        z_g = 0
        new_cap = cap - sum([w[j] for j in subset])
        X = []
        for j in range(0,n_items):
            if not (j in subset) and w[j] <= new_cap:
                z_g = z_g + p[j]
                new_cap = new_cap - w[j]
                X.append(j)
            ops_ = ops_+1
        return z_g, X, ops_
    
    z_h = 0
    X_h = []

    for subset in powerset_k(seq = range(n_items)):
        if sum([w[j] for j in subset]) <= cap:

            z_g, X, gs_ops= GS(subset)
            ops = ops + gs_ops

            if z_g + sum([p[j] for j in subset]) > z_h:
                z_h = z_g + sum([p[j] for j in subset])
                X_h = list(set(X + list(subset)))
    
    idx = np.zeros(n_items, dtype=int)
    idx[X_h] = 1
    return list(idx), z_h, ops

In [7]:
def branch_and_bound(cap, p, w):
    '''Branсhes and bounds method'''
    class Node():
        def __init__(self, level, items, w, p):
            self.level = level
            self.items = items
            self.weight = w
            self.profit = p

    items = sorted(range(len(w)), key=lambda i:p[i]/w[i], reverse=True)
    ops = int(len(w) * (np.log2(len(w)))) # Complexity O(N*log(N)) of TimSort (python Sorted method algorithm)
    best_items = []
    
    def get_ub(node):
        ops = 0
        cur_cap = cap - node.weight
        bound = node.profit
 
        for i in range(node.level+1, len(items)):
            item = items[i]
            ops = ops+1
            if cur_cap >= w[item]:
                bound = bound + p[item]
                cur_cap = cur_cap - w[item]
            else:
                bound += cur_cap * p[item]/w[item]
                break
 
        return bound, ops

    Q = [Node(-1, [], 0, 0)]
    profit = 0
    
    while Q:
        v = Q.pop(0)
        
        ub, ops_ = get_ub(v)
        ops = ops + ops_

        if ub > profit and v.level < len(items)-1:
            next_level = v.level+1
            wi = Node(next_level, v.items+[items[next_level]],
                      v.weight+w[items[next_level]],
                      v.profit+p[items[next_level]])
            
            if wi.weight <= cap and wi.profit > profit:
                profit = wi.profit
                best_items = wi.items
            
            ub, ops_ = get_ub(wi)
            ops = ops + ops_

            if ub > profit:
                Q.append(wi)

            wo = Node(next_level, v.items,
                      v.weight,
                      v.profit)
            
            ub, ops_ = get_ub(wo)
            ops = ops + ops_

            if ub > profit:
                Q.append(wo)

    idxs = np.zeros(len(w), dtype = int)
    idxs[best_items]=1

    return list(idxs), profit, ops

In [8]:
files_list = []
for i in range(1, 8):
    files_list.append((f'./benchmarks/p0{i}_c.txt', f'./benchmarks/p0{i}_w.txt', f'./benchmarks/p0{i}_p.txt', f'./benchmarks/p0{i}_s.txt'))

In [24]:
dp_stats = []
approx_stats = []
ptas_stats = []
bb_stats = []

In [25]:
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        profits = list(map(int, p.read().split()))
        answer = list(map(int, s.read().split()))
        print(f'Sample {i + 1}')
        
        print('--------------------')
        print('DP on weights:')
        indexes, profit, operations, time = knapsack(method=dp, cap=capacity, p=profits, w=weights)
        dp_stats.append([f'sample{i+1}', operations, time])
        weight = count_weight(indexes, weights)
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        assert answer == indexes
        print('--------------------')
        print('2-approx algorithm:')
        indexes, profit, operations, time = knapsack(method=two_approx, cap=capacity, p=profits, w=weights)
        approx_stats.append([f'sample{i+1}', operations, time])
        weight = count_weight(indexes, weights)
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        print('--------------------')
        print('PTAS:')
        indexes, profit, operations, time = knapsack(method=ptas, cap=capacity, p=profits, w=weights)
        ptas_stats.append([f'sample{i+1}', operations, time])
        weight = count_weight(indexes, weights)
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        print('--------------------')
        print('B&B:')
        indexes, profit, operations, time = knapsack(method=branch_and_bound, cap=capacity, p=profits, w=weights)
        bb_stats.append([f'sample{i+1}', operations, time])
        weight = count_weight(indexes, weights)
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        print('--------------------')
        print(f'Optimal answer: {answer}')
        
        print('\n\n')

Sample 1
--------------------
DP on weights:
Capacity: 165
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Time: 0.0006998062133789062
Operations count: 1650
--------------------
2-approx algorithm:
Capacity: 165
Items: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
Profit: 266
Weight: 127
Time: 0.0
Operations count: 94
--------------------
PTAS:
Capacity: 165
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Time: 0.0008501529693603516
Operations count: 1420
--------------------
B&B:
Capacity: 165
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Time: 0.0002496957778930664
Operations count: 261
--------------------
Optimal answer: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]



Sample 2
--------------------
DP on weights:
Capacity: 26
Items: [0, 1, 1, 1, 0]
Profit: 51
Weight: 26
Time: 4.9948692321777344e-05
Operations count: 130
--------------------
2-approx algorithm:
Capacity: 26
Items: [1, 0, 1, 0, 0]
Profit: 47
Weight: 23
Time: 0.0
Operations count: 38
-------------------

In [35]:
capacities = []
lens = []

In [36]:
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        capacities.append(capacity)
        lens.append(len(weights))

In [39]:
import pandas as pd
#dp stats
df = pd.DataFrame(data=dp_stats, columns=['Набор данных', 'Количество операций', 'Время'])
df['Емкость рюкзака'] = capacities
df['Количество предметов в сэмпле'] = lens
df = df.reset_index(drop=True)
df[['Набор данных', 'Емкость рюкзака', 'Количество предметов в сэмпле', 'Количество операций', 'Время']]

,Набор данных,Емкость рюкзака,Количество предметов в сэмпле,Количество операций,Время
0,sample1,165,10,1650,0.00070
1,sample2,26,5,130,0.00005
2,sample3,190,6,1140,0.00045
3,sample4,50,7,350,0.00010
4,sample5,104,8,832,0.00035
5,sample6,170,7,1190,0.00050
6,sample7,750,15,11250,0.00510


In [40]:
#2-approx stats
df = pd.DataFrame(data=approx_stats, columns=['Набор данных', 'Количество операций', 'Время'])
df['Емкость рюкзака'] = capacities
df['Количество предметов в сэмпле'] = lens
df = df.reset_index(drop=True)
df[['Набор данных', 'Емкость рюкзака', 'Количество предметов в сэмпле', 'Количество операций', 'Время']]

,Набор данных,Емкость рюкзака,Количество предметов в сэмпле,Количество операций,Время
0,sample1,165,10,94,0.00000
1,sample2,26,5,38,0.00000
2,sample3,190,6,48,0.00000
3,sample4,50,7,57,0.00000
4,sample5,104,8,70,0.00005
5,sample6,170,7,59,0.00000
6,sample7,750,15,161,0.00005


In [41]:
#ptas stats
df = pd.DataFrame(data=ptas_stats, columns=['Набор данных', 'Количество операций', 'Время'])
df['Емкость рюкзака'] = capacities
df['Количество предметов в сэмпле'] = lens
df = df.reset_index(drop=True)
df[['Набор данных', 'Емкость рюкзака', 'Количество предметов в сэмпле', 'Количество операций', 'Время']]

,Набор данных,Емкость рюкзака,Количество предметов в сэмпле,Количество операций,Время
0,sample1,165,10,1420,0.00085
1,sample2,26,5,90,0.00010
2,sample3,190,6,204,0.00010
3,sample4,50,7,497,0.00020
4,sample5,104,8,1640,0.00060
5,sample6,170,7,420,0.00020
6,sample7,750,15,245220,0.06325


In [42]:
#branches and bounds stats
df = pd.DataFrame(data=bb_stats, columns=['Набор данных', 'Количество операций', 'Время'])
df['Емкость рюкзака'] = capacities
df['Количество предметов в сэмпле'] = lens
df = df.reset_index(drop=True)
df[['Набор данных', 'Емкость рюкзака', 'Количество предметов в сэмпле', 'Количество операций', 'Время']]

,Набор данных,Емкость рюкзака,Количество предметов в сэмпле,Количество операций,Время
0,sample11,165,10,261,0.00025
1,sample12,26,5,67,0.00010
2,sample13,190,6,109,0.00015
3,sample14,50,7,164,0.00025
4,sample15,104,8,241,0.00030
5,sample16,170,7,300,0.00040
6,sample17,750,15,8787,0.01560
